<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Transformers_con_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Transformers con Keras

Este notebook demuestra el uso de Transformers para análisis de sentimiento en español.
Los Transformers son especialmente poderosos porque:
1. Procesan todas las palabras en paralelo (a diferencia de las RNN)
2. Pueden captar relaciones entre palabras distantes fácilmente
3. Son más rápidos de entrenar que las RNN

##1) Preparación de Datos

Usamos frases simples en español para demostrar el concepto

In [7]:
# Import necesarios
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Datos de ejemplo
sentences = [
    'Me gusta mucho este curso',
    'Estoy aburrido de la rutina diaria',
    'El clima hoy está maravilloso',
    'No estoy satisfecho con el servicio',
]
labels = [1, 0, 1, 0]  # 1: Positivo, 0: Negativo

##2) Preprocesamiento del Texto

Convertimos las palabras a números usando un tokenizador

In [ ]:
# Preparar el texto
vocab_size = 1000
max_length = 10

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(sequences, maxlen=max_length)

# Convertir a tensores de TensorFlow
X = tf.convert_to_tensor(X, dtype=tf.float32)
labels = tf.convert_to_tensor(labels, dtype=tf.float32)

##2) Creacion del modelo Transformer

El modelo tiene las siguientes capas clave:
1. Embedding: Convierte palabras en vectores
2. Multi-Head Attention: La "magia" del Transformer que permite procesar
   relaciones entre palabras
3. Layer Normalization: Ayuda al entrenamiento
4. Dense: Capa final para clasificación

In [10]:
# Crear modelo Transformer simplificado
def create_transformer_classifier(vocab_size, max_length):
    inputs = Input(shape=(max_length,))

    # Capa de Embedding
    embedding_layer = tf.keras.layers.Embedding(vocab_size, 32)(inputs)

    # Multi-Head Attention (la parte mágica del Transformer)
    attention = MultiHeadAttention(
        num_heads=2, key_dim=32
    )(embedding_layer, embedding_layer, embedding_layer)

    # Normalización y Residual connection
    x = LayerNormalization()(attention + embedding_layer)

    # Clasificación final
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = Dense(1, activation='sigmoid')(x)

    return Model(inputs, outputs)

# Crear y compilar modelo
model = create_transformer_classifier(vocab_size, max_length)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 10, 32)         │         32,000 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_3    │ (None, 10, 32)         │          8,416 │ embedding_3[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_3[0][0],     │
│                           │                        │                │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 10, 32)         │              0 │ multi_head_attention_… │
│                           │                        │                │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_3     │ (None, 10, 32)         │             64 │ add_3[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 32)             │              0 │ layer_normalization_3… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │             33 │ global_average_poolin… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 40,513 (158.25 KB)

 Trainable params: 40,513 (158.25 KB)

 Non-trainable params: 0 (0.00 B)

##2) Entrenamiento y Ejecución

Entrenamos el modelo y probamos con nuevas frases

In [11]:
# Entrenar modelo
history = model.fit(
    X, labels,
    epochs=10,
    batch_size=2,
    verbose=1
)

# Probar con nuevas frases
test_sentences = [
    "Este producto es excelente",
    "El servicio fue terrible"
]

# Preparar texto de prueba
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length)
test_padded = tf.convert_to_tensor(test_padded, dtype=tf.float32)

# Realizar predicciones
predictions = model.predict(test_padded)

# Mostrar resultados
for sentence, prediction in zip(test_sentences, predictions):
    sentiment = "Positivo" if prediction > 0.5 else "Negativo"
    print(f"Frase: '{sentence}'")
    print(f"Sentimiento: {sentiment} (probabilidad: {prediction[0]:.2f})")
    print()

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6667 - loss: 0.7396
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3333 - loss: 0.7001     
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.5730 
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.5075 
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.4847
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.3941 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.3503
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.2958 
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.2677
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.2294 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Frase: 'Este producto es excelente'
Sentimiento: Positivo (probabilidad: 0.69)

Frase: 'El servicio fue terrible'
Sentimiento: